# <span style="color: green; font-size: 30px; font-weight: bold;">redaing the data</span>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv(r'E:\ARAS\customer churn paper\code\Aras_transactions_new.csv')
df

,date,SaleTime,art_name,articles_id,price_purchase,price_item,Quantity,BranchCode,BranchName,customers_id,takhfif,gerd,Maliat,Payment channel,Sales channel,factor_id,d_dat,group_id
0,1395-01-01,23:59:58,خانواده پپسي,611,2500.0,2500,1.0,13803,پاسداران,9120306517,0.0,-100.0,3600.0,نقدی,حضوری,6,2016-03-20,600
1,1395-01-01,23:59:58,سيب زميني مخصوص,501,7500.0,7500,1.0,13803,پاسداران,9120306517,0.0,-100.0,3600.0,نقدی,حضوری,6,2016-03-20,500
2,1395-01-01,23:59:58,لند برگر,101,27000.0,13500,2.0,13803,پاسداران,9120306517,0.0,-100.0,3600.0,نقدی,حضوری,6,2016-03-20,100
3,1395-01-01,22:36:01,اسموكي برگر,102,14500.0,14500,1.0,13804,اندرزگو,9128302458,0.0,-205.0,2205.0,نقدی,حضوری,9,2016-03-20,100
4,1395-01-01,22:36:01,سيب زميني مخصوص,501,7500.0,7500,1.0,13804,اندرزگو,9128302458,0.0,-205.0,2205.0,نقدی,حضوری,9,2016-03-20,500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7284203,1403-05-12,15:51:15,چيکن رپ,903,194500.0,194500,1.0,13814,شعبه بازار (پرنسا),9351555271,0.0,50.0,19450.0,نقدی,حضوری,3765492,2024-08-02,900
7284204,1403-05-12,15:51:15,زيرو قوطي,604,24500.0,24500,1.0,13814,شعبه بازار (پرنسا),9351555271,0.0,50.0,19450.0,نقدی,حضوری,3765492,2024-08-02,600
7284205,1403-05-12,16:23:22,لند برگر,101,242500.0,242500,1.0,13814,شعبه بازار (پرنسا),9173058935,0.0,0.0,24250.0,نقدی,حضوری,3765493,2024-08-02,100
7284206,1403-05-12,16:23:22,هي دي استوايي,641,30000.0,30000,1.0,13814,شعبه بازار (پرنسا),9173058935,0.0,0.0,24250.0,نقدی,حضوری,3765493,2024-08-02,600


# <span style="color: green; font-size: 30px; font-weight: bold;">filtering</span>

In [3]:
def process_dataframe(df):
    # Select specific columns
    df = df[['customers_id', 'price_purchase', 'd_dat', 'group_id']]
    
    # Convert 'd_dat' to datetime format
    df['date'] = pd.to_datetime(df['d_dat'])
    
    # Drop the original 'd_dat' column
    df = df.drop('d_dat', axis=1)
    
    # Extract the day of the week from 'date'
    df['day_of_week'] = df['date'].dt.day_name()
    
    # Convert 'customers_id' to string
    df['customers_id'] = df['customers_id'].astype(str)
    
    # Filter rows where 'customers_id' starts with '9' and its length is between 8 and 13
    df = df[df['customers_id'].str.startswith('9') & df['customers_id'].str.len().between(8, 13)]
    
    # Convert 'customers_id' and 'group_id' back to appropriate types
    df['customers_id'] = df['customers_id'].astype('int64').astype(str)
    df['group_id'] = df['group_id'].astype(str)
    
    return df

df = process_dataframe(df)


In [4]:
unique_customer = df['customers_id'].nunique()
print(f"number of unique customers in the original data = {unique_customer}")

number of unique customers in the original data = 864869


# <span style="color: green; font-size: 30px; font-weight: bold;">creating validation dataframe</span>

Time calculator 

In [5]:
# Use this function to estimate the average number of days in a window if neccesary 

def average_days_in_months(months):
    # Average days per month considering leap years
    average_days_per_month = (365 * 3 + 366) / (12 * 4)
    
    # Total days for the given number of months
    total_days = average_days_per_month * months
    
    return round(total_days)

given_month = 7   # Enter the number of months as an input to the function 

days_in_months = average_days_in_months(given_month)
print(f"Average number of days in {given_month} months: {days_in_months}")

Average number of days in 7 months: 213


In [5]:
import pandas as pd

def create_dataframes(date_start, length_of_each_window, gap_between_each_dataframe, num_df, df):
    # Determine the initial start date
    date1_start = pd.Timestamp(date_start)
    date2_start = df['date'].min()

    # Calculate the difference in days to adjust the start date
    difference_in_days_start = (date1_start - date2_start).days
    start_cutoff_date_0 = df['date'].min() + pd.Timedelta(days=difference_in_days_start)
    break_cutoff_date_0 = start_cutoff_date_0 + pd.Timedelta(days=length_of_each_window)

    # Initialize lists to store the start and break dates
    start_cutoff_dates = [start_cutoff_date_0]
    break_cutoff_dates = [break_cutoff_date_0]

    # Calculate the subsequent dates for each DataFrame
    for i in range(num_df - 1):
        next_start_date = start_cutoff_dates[i] + pd.Timedelta(days=gap_between_each_dataframe)
        next_break_date = break_cutoff_dates[i] + pd.Timedelta(days=gap_between_each_dataframe)
        start_cutoff_dates.append(next_start_date)
        break_cutoff_dates.append(next_break_date)
        

    # Create empty DataFrames
    dataframes = [pd.DataFrame() for _ in range(num_df)]

    # Filter the DataFrame based on start and break cutoff dates
    for i in range(len(break_cutoff_dates)):
        dataframes[i] = df[(df['date'] >= start_cutoff_dates[i]) & (df['date'] <= break_cutoff_dates[i])]
        globals()[f"df{i}"] = dataframes[i]

    return start_cutoff_dates, break_cutoff_dates, dataframes

length_of_each_window = 213 # Data window length             
length_of_the_label_window = 30 # Label window length 
gap_between_each_dataframe = length_of_each_window + length_of_the_label_window # Data window + Label window 

num_df = 1

total_gap = num_df * gap_between_each_dataframe

# Calculate the new start date
date_start = df['date'].max() - pd.Timedelta(days=total_gap)

start_cutoff_dates, break_cutoff_dates, dfs = create_dataframes(date_start, length_of_each_window, gap_between_each_dataframe, num_df, df)

# Assign the data frames
df_valid = dfs[0]

len(dfs)

1

<span style="color: red; font-size: 25px; font-weight: bold;">extracting labels for each dataframe from the original df</span>

In [6]:
import pandas as pd

def create_label_dfs(df, break_cutoff_dates, length_of_the_label_window, num_df):
    label_dfs = []
    label_lists = []

    # Loop through each DataFrame and create the corresponding label DataFrame
    for i in range(num_df):
        df_label = df[(df['date'] >= break_cutoff_dates[i]) & 
                      (df['date'] <= break_cutoff_dates[i] + pd.Timedelta(days=length_of_the_label_window))]
        label_dfs.append(df_label)
        label_lists.append(df_label['customers_id'].tolist())

        # Dynamically create a variable name and assign the DataFrame to it
        globals()[f'df{i}_label'] = df_label
    
    return label_dfs, label_lists


length_of_the_label_window = length_of_the_label_window  # Set the length of the label window as needed
num_df = num_df  # Number of DataFrames

label_dfs, label_lists = create_label_dfs(df, break_cutoff_dates, length_of_the_label_window, num_df)

df_valid_label = label_dfs[0]

df_valid_label_list = label_lists[0]


In [7]:
df_valid['date'].min(), df_valid['date'].max() , df_valid_label['date'].min(), df_valid_label['date'].max()

(Timestamp('2023-12-03 00:00:00'),
 Timestamp('2024-07-03 00:00:00'),
 Timestamp('2024-07-03 00:00:00'),
 Timestamp('2024-08-02 00:00:00'))

<span style="color: red; font-size: 25px; font-weight: bold;">dropping duplicate rows and also consider only one of customers purchase for each date. dropping 2 or less transactions</span>

In [8]:
def clean_dataframe(df):
    # Drop duplicates based on 'customers_id' and 'date', keeping the first occurrence
    df_cleaned = df.drop_duplicates(subset=['customers_id', 'date'], keep='first')

    # Identify customers with two or fewer purchases
    customers_to_drop = df_cleaned['customers_id'].value_counts()[df_cleaned['customers_id'].value_counts() <= 2].index

    # Drop these customers from the DataFrame
    df_cleaned = df_cleaned[~df_cleaned['customers_id'].isin(customers_to_drop)]

    return df_cleaned


df_valid = clean_dataframe(df_valid)


# <span style="color: green; font-size: 30px; font-weight: bold;">adding feature</span>

## lifetime feature 

In [9]:
import lifetimes

# Function to calculate metrics
def calculate_metrics(df, current_date):
    metrics_pd = (
        lifetimes.utils.summary_data_from_transaction_data(
            df,
            customer_id_col='customers_id',
            datetime_col='date',
            observation_period_end=current_date, 
            freq='D'
        )
    )
    filtered_pd = metrics_pd[metrics_pd['frequency'] > 0]
    filtered_pd.reset_index(inplace=True)  # Ensure customers_id is a column
    return filtered_pd

# Calculate metrics for each subset with different current_date values
metrics_valid = calculate_metrics(df_valid, df_valid['date'].max())


In [10]:
from lifetimes import BetaGeoFitter

# Function to fit BG/NBD model and predict expected number of purchases in the next 30 days
def predict_expected_purchases(metrics_pd, t=30, penalizer_coef=0.01):
    bgf = BetaGeoFitter(penalizer_coef=penalizer_coef)
    bgf.fit(metrics_pd['frequency'], metrics_pd['recency'], metrics_pd['T'])
    
    # Predict the expected number of purchases for the next t days
    metrics_pd[f'expected_purchases_{t}'] = bgf.predict(t, metrics_pd['frequency'], metrics_pd['recency'], metrics_pd['T'])
    return metrics_pd

# Fit BG/NBD model and predict expected number of purchases in the next 30 days
metrics_valid = predict_expected_purchases(metrics_valid, t=length_of_the_label_window, penalizer_coef=0.1)


In [11]:
t = length_of_the_label_window
df_valid = pd.merge(df_valid, metrics_valid[['customers_id', 'recency', f'expected_purchases_{t}']], on='customers_id', how='left')

## own features

In [12]:
import pandas as pd
import numpy as np

def generate_features(df, df_name=None, break_cutoff_dates=None):
    # Step 1: Add 'purchase_month' feature
    df['purchase_month'] = df['date'].dt.to_period('M')
    
    # Step 2: Add 'most_common_day' feature
    def add_most_common_day(df):
        def most_common_day(series):
            return series.mode().iloc[0]
        common_days = df.groupby('customers_id')['day_of_week'].agg(most_common_day).reset_index()
        common_days.rename(columns={'day_of_week': 'most_common_day'}, inplace=True)
        return df.merge(common_days, on='customers_id')

    df = add_most_common_day(df)
    
    # Step 3: Add 'number_of_purchases' feature
    def purchase_count(df):
        count_purchase = df['customers_id'].value_counts().rename('number_of_purchases')
        return df.merge(count_purchase, left_on='customers_id', right_index=True)

    df = purchase_count(df)
    
    # Step 4: Calculate 'weighted_mean_time_between_purchases'
    def calculate_weighted_mean_time_between_purchases(df):
        sorted_df = df.sort_values(by=['customers_id', 'date'], ascending=[True, False])
        sorted_df['time_diff'] = sorted_df.groupby('customers_id')['date'].diff().dt.days * -1
        time_diffs = sorted_df.dropna(subset=['time_diff'])
        
        def weighted_mean(group):
            n = len(group)
            weights = [(n - i) ** 2 for i in range(n)]
            return np.average(group, weights=weights)

        weighted_mean_time_diff = time_diffs.groupby('customers_id')['time_diff'].apply(weighted_mean).reset_index(name='weighted_mean_time_between_purchases')
        return df.merge(weighted_mean_time_diff, on='customers_id', how='left').fillna(0)

    df = calculate_weighted_mean_time_between_purchases(df)
    
    # Step 5: Calculate 'std_between_purchase'
    def calculate_std_features(df):
        sorted_df = df.sort_values(by=['customers_id', 'date'], ascending=[True, False])
        sorted_df['time_diff'] = sorted_df.groupby('customers_id')['date'].diff().dt.days * -1

        def calculate_std_between_purchase(group):
            n = len(group)
            if n == 1:
                return 0
            weights = [(n - i) ** 2 for i in range(1, n)]
            weight_sum = sum(weights)
            normalized_weights = [w / weight_sum for w in weights]
            weighted_mean = np.dot(group[1:], normalized_weights)
            squared_differences = ((group[1:] - weighted_mean) ** 2)
            return np.sqrt(np.dot(squared_differences, normalized_weights))

        std_between_purchase = sorted_df.groupby('customers_id')['time_diff'].apply(calculate_std_between_purchase).reset_index(name='std_between_purchase')
        return df.merge(std_between_purchase, on='customers_id', how='left')

    df = calculate_std_features(df)
    
    # Step 6: Calculate 'max_time_without_purchase'
    def calculate_max_without_purchase(df):
        sorted_df = df.sort_values(by=['customers_id', 'date'], ascending=[True, False])
        sorted_df['time_diff'] = sorted_df.groupby('customers_id')['date'].diff().dt.days * -1
        max_time_without_purchase = sorted_df.groupby('customers_id')['time_diff'].max().reset_index(name='max_time_without_purchase')
        return df.merge(max_time_without_purchase, on='customers_id', how='left')

    df = calculate_max_without_purchase(df)
    
    # Time since last purchase in months 
    # # Step 7: Conditional 'calculate_time_since_last_purchase'
    # def calculate_time_since_last_purchase(df, index):
    #     last_purchase_date = df.groupby('customers_id')['date'].max().reset_index(name='last_purchase_date')
    #     specific_date = break_cutoff_dates[index]
    #     last_purchase_date['time_since_last_purchase'] = (
    #         (specific_date.year - last_purchase_date['last_purchase_date'].dt.year) * 12 +
    #         (specific_date.month - last_purchase_date['last_purchase_date'].dt.month)
    #     )
    #     return df.merge(last_purchase_date[['customers_id', 'time_since_last_purchase']], on='customers_id', how='left')
    
    # Time since last purchase in days 
    def calculate_time_since_last_purchase(df, index):
        # Get the last purchase date for each customer
        last_purchase_date = df.groupby('customers_id')['date'].max().reset_index(name='last_purchase_date')
        
        # Define the specific date (cutoff date)
        specific_date = break_cutoff_dates[index]
        
        # Calculate the difference in days
        last_purchase_date['time_since_last_purchase'] = (specific_date - last_purchase_date['last_purchase_date']).dt.days
        
        # Merge the result back into the original DataFrame
        return df.merge(last_purchase_date[['customers_id', 'time_since_last_purchase']], on='customers_id', how='left')
    
    
    
    
    # Apply the correct time_since_last_purchase calculation based on df_name
    if df_name is not None:
        index_mapping = {'df_valid': 0}
        if df_name in index_mapping:
            df = calculate_time_since_last_purchase(df, index_mapping[df_name])

    # Step 8: Conditional 'calculate_time_since_first_purchase'
    def calculate_time_since_first_purchase(df, index):
        first_purchase_date = df.groupby('customers_id')['date'].min().reset_index(name='first_purchase_date')
        specific_date = break_cutoff_dates[index]
        first_purchase_date['time_since_first_purchase'] = (specific_date - first_purchase_date['first_purchase_date']).dt.days
        return df.merge(first_purchase_date[['customers_id', 'time_since_first_purchase']], on='customers_id', how='left')

    # Apply the correct time_since_first_purchase calculation based on df_name
    if df_name is not None:
        if df_name in index_mapping:
            df = calculate_time_since_first_purchase(df, index_mapping[df_name])

    # Step 9: Calculate 'transaction recency'
    def calculate_transaction_recency(df):
        num_unique_customers = df['customers_id'].nunique()
        last_purchase_date = df.groupby('customers_id')['date'].max().reset_index(name='last_purchase_date')
        last_purchase_date['time_since_last_purchase'] = df['time_since_last_purchase']
        last_purchase_date['transaction recency'] = last_purchase_date['time_since_last_purchase'] / num_unique_customers
        return df.merge(last_purchase_date[['customers_id', 'transaction recency']], on='customers_id', how='left')

    df = calculate_transaction_recency(df)
    
    # Step 10: Add one-hot encoded features for 'time_since_last_purchase'
    def add_onehot_last_purchase(df, threshold):
        upper_limit = threshold + 30
        new_column_name = f'last_between_{threshold}_and_{upper_limit}'
        df[new_column_name] = ((df['time_since_last_purchase'] >= threshold) & (df['time_since_last_purchase'] < upper_limit)).astype(int)
        return df

    thresholds = [0, 30, 60]
    for threshold in thresholds:
        df = add_onehot_last_purchase(df, threshold)
    
    # Step 11: Calculate 'diff_last_and_penultimate'
    def calculate_time_between_last_two_purchases(df):
        if df['date'].dtype == 'object':
            df['date'] = pd.to_datetime(df['date'])
        df_sorted = df.sort_values(by=['customers_id', 'date'])
        grouped_dates = df_sorted.groupby('customers_id')['date'].apply(list)
        diff_last_and_penultimate = grouped_dates.apply(lambda dates: (dates[-1] - dates[-2]).days if len(dates) > 1 else None)
        diff_last_and_penultimate = diff_last_and_penultimate.reset_index(name='diff_last_and_penultimate')
        return df.merge(diff_last_and_penultimate, on='customers_id', how='left')

    df = calculate_time_between_last_two_purchases(df)
    
    # Step 12: Calculate 'diff_penultimate_and_previous'
    def calculate_time_between_penultimate_and_previous(df):
        if df['date'].dtype == 'object':
            df['date'] = pd.to_datetime(df['date'])
        df_sorted = df.sort_values(by=['customers_id', 'date'])  # Corrected line
        grouped_dates = df_sorted.groupby('customers_id')['date'].apply(list)
        diff_penultimate_and_previous = grouped_dates.apply(lambda dates: (dates[-2] - dates[-3]).days if len(dates) > 2 else None)
        diff_penultimate_and_previous = diff_penultimate_and_previous.reset_index(name='diff_penultimate_and_previous')
        return df.merge(diff_penultimate_and_previous, on='customers_id', how='left')

    df = calculate_time_between_penultimate_and_previous(df)

    
    # Step 13: Add 'threshold' features
    def add_threshold_features(df):
        h_1 = 2
        h_2 = 4
        h_3 = 8
        df['tresh_1'] = df['weighted_mean_time_between_purchases'] + h_1 * df['std_between_purchase']
        df['tresh_2'] = df['weighted_mean_time_between_purchases'] + h_2 * df['std_between_purchase']
        df['tresh_3'] = df['weighted_mean_time_between_purchases'] + h_3 * df['std_between_purchase']
        return df

    df = add_threshold_features(df)

    # Step 14: Frequency classification
    def classify_frequency(row):
        if row['time_since_last_purchase'] <= row['tresh_1']:
            return 'normal'
        elif row['time_since_last_purchase'] <= row['tresh_2']:
            return 'attrition'
        elif row['time_since_last_purchase'] <= row['tresh_3']:
            return 'at-risk'
        else:
            return 'lost'

    def frequency_classification(df):
        df['freq_class'] = df.apply(classify_frequency, axis=1)
        return df

    df = frequency_classification(df)

    # Step 15: Add moving average features
    def add_moving_average_purchase_feature(df, window_size, column_name):
        df_sorted = df.sort_values(by=['customers_id', 'date']).copy()
        df[column_name] = df_sorted.groupby('customers_id')['price_purchase'].transform(
            lambda x: x.rolling(window=window_size, min_periods=1).mean())
        return df

    df = add_moving_average_purchase_feature(df, 3, 'rolling_avg_3')
    df = add_moving_average_purchase_feature(df, 6, 'rolling_avg_6')

    # Step 16: Add binned purchase feature
    def add_binned_purchase_feature(df):
        purchase_sums = df.groupby('customers_id')['price_purchase'].sum().reset_index(name='binned_purchase')
        return pd.merge(df, purchase_sums, on='customers_id', how='left')

    df = add_binned_purchase_feature(df)

    # Step 17: Calculate relative change
    def calculate_relative_change(df):
        df_sorted = df.sort_values(by=['customers_id', 'date'])
        if 'rolling_avg_6' not in df.columns:
            df_sorted['rolling_avg_6'] = df_sorted.groupby('customers_id')['price_purchase'].transform(
                lambda x: x.rolling(window=6, min_periods=1).mean())
        df_sorted['rolling_avg_6'].fillna(0, inplace=True)
        last_price_purchase = df_sorted.groupby('customers_id')['price_purchase'].last().fillna(0)
        sixth_last_rolling_avg = df_sorted.groupby('customers_id').apply(
            lambda x: x['rolling_avg_6'].iloc[-6] if len(x) >= 6 else x['rolling_avg_6'].iloc[0]).fillna(0)
        relative_change = (last_price_purchase - sixth_last_rolling_avg) / sixth_last_rolling_avg.replace(0, 1)
        df['d'] = df['customers_id'].map(relative_change)
        return df

    df = calculate_relative_change(df)

    # Step 18: Purchase trend classification
    def purchase_trend_classifier(row):
        a1 = 0.15
        a2 = 0.225
        a3 = 0.3
        if row['d'] <= -a3:
            return 'decreasing--'
        elif -a3 < row['d'] <= -a2:
            return 'decreasing-'
        elif -a2 < row['d'] <= -a1:
            return 'decreasing'
        elif -a1 < row['d'] <= a1:
            return 'stable'
        elif a1 < row['d'] <= a2:
            return 'increasing'
        elif a2 < row['d'] <= a3:
            return 'increasing+'
        elif a3 < row['d']:
            return 'increasing++'

    def purchase_trend(df):
        df['purchase_trend'] = df.apply(purchase_trend_classifier, axis=1)
        return df

    df = purchase_trend(df)

    # Step 19: Add purchase value features (max, mean, median)
    def add_max_purchase_feature(df):
        max_purchase = df.groupby('customers_id')['price_purchase'].max().reset_index(name='max_purchase')
        return pd.merge(df, max_purchase, on='customers_id', how='left')

    def add_mean_purchase_feature(df):
        mean_purchase = df.groupby('customers_id')['price_purchase'].mean().reset_index(name='mean_purchase')
        return pd.merge(df, mean_purchase, on='customers_id', how='left')

    def add_median_purchase_feature(df):
        median_purchase = df.groupby('customers_id')['price_purchase'].median().reset_index(name='median_purchase')
        return pd.merge(df, median_purchase, on='customers_id', how='left')

    df = add_max_purchase_feature(df)
    df = add_mean_purchase_feature(df)
    df = add_median_purchase_feature(df)

    # Step 20: Calculate relative purchase value
    def calculate_relative_purchase_value(df):
        df = df.sort_values(['customers_id', 'date'])
        rel_purchase_values = df.groupby('customers_id').apply(lambda group: calculate_rel_value(group))
        return df.merge(rel_purchase_values.rename('rel_purchase_value'), on='customers_id', how='left')

    def calculate_rel_value(group):
        if len(group) < 5:
            first_value = group.iloc[0]['price_purchase']
            last_value = group.iloc[-1]['price_purchase']
            if first_value == 0:
                return np.nan
            return (last_value - first_value) / first_value
        else:
            fifth_from_last_value = group.iloc[-5]['price_purchase']
            last_value = group.iloc[-1]['price_purchase']
            if fifth_from_last_value == 0:
                return np.nan
            return (last_value - fifth_from_last_value) / fifth_from_last_value

    df = calculate_relative_purchase_value(df)
    df['rel_purchase_value'].fillna(0, inplace=True)

    # Step 21: Time frame classification
    def time_frame_var_row(row):
        mu = 0.3
        if row['rel_purchase_value'] < -mu:
            return 'steady'
        elif abs(row['rel_purchase_value']) <= mu:
            return 'within-limits'
        elif row['rel_purchase_value'] > mu:
            return 'altering'

    def time_frame_char(df):
        df['time_frame_char'] = df.apply(time_frame_var_row, axis=1)
        return df

    df = time_frame_char(df)

    # Step 22: Add year, month, and day features
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    df['day'] = df['date'].dt.day
    
    return df

# Example usage:
df_valid = generate_features(df_valid, df_name='df_valid', break_cutoff_dates=break_cutoff_dates)

<span style="color: red; font-size: 25px; font-weight: bold;">make a list of features </span>

In [13]:
features_list = df_valid.columns.tolist()
print(features_list)
len(features_list)

['customers_id', 'price_purchase', 'group_id', 'date', 'day_of_week', 'recency', 'expected_purchases_30', 'purchase_month', 'most_common_day', 'number_of_purchases', 'weighted_mean_time_between_purchases', 'std_between_purchase', 'max_time_without_purchase', 'time_since_last_purchase', 'time_since_first_purchase', 'transaction recency', 'last_between_0_and_30', 'last_between_30_and_60', 'last_between_60_and_90', 'diff_last_and_penultimate', 'diff_penultimate_and_previous', 'tresh_1', 'tresh_2', 'tresh_3', 'freq_class', 'rolling_avg_3', 'rolling_avg_6', 'binned_purchase', 'd', 'purchase_trend', 'max_purchase', 'mean_purchase', 'median_purchase', 'rel_purchase_value', 'time_frame_char', 'year', 'month', 'day']


38

In [14]:
df_valid.shape

(177390, 38)

# <span style="color: green; font-size: 30px; font-weight: bold;">adding label</span>

In [15]:
def label_df_with_list(df, customer_ids_list):
    # Create a new DataFrame to avoid modifying the original one
    labeled_df = df.copy()
    
    # Create a new column 'label' in labeled_df and initialize it with 0
    labeled_df['label'] = 0
    
    # Set label as 1 for customer IDs present in customer_ids_list
    labeled_df.loc[labeled_df['customers_id'].isin(customer_ids_list), 'label'] = 1
    
    return labeled_df

df_valid_labled = label_df_with_list(df_valid, df_valid_label_list)

In [16]:
print(df_valid.columns)

Index(['customers_id', 'price_purchase', 'group_id', 'date', 'day_of_week',
       'recency', 'expected_purchases_30', 'purchase_month', 'most_common_day',
       'number_of_purchases', 'weighted_mean_time_between_purchases',
       'std_between_purchase', 'max_time_without_purchase',
       'time_since_last_purchase', 'time_since_first_purchase',
       'transaction recency', 'last_between_0_and_30',
       'last_between_30_and_60', 'last_between_60_and_90',
       'diff_last_and_penultimate', 'diff_penultimate_and_previous', 'tresh_1',
       'tresh_2', 'tresh_3', 'freq_class', 'rolling_avg_3', 'rolling_avg_6',
       'binned_purchase', 'd', 'purchase_trend', 'max_purchase',
       'mean_purchase', 'median_purchase', 'rel_purchase_value',
       'time_frame_char', 'year', 'month', 'day'],
      dtype='object')


In [17]:
len(df_valid.columns)

38

In [18]:
print(set(df_valid_labled['label']))

{0, 1}


# <span style="color: green; font-size: 30px; font-weight: bold;">preparing the data for the model</span>

In [22]:
# # List of DataFrames
# dfs = [df_valid_labled]

# # Concatenate all DataFrames
# concat_df = pd.concat(dfs, ignore_index=True)

# concat_df

,customers_id,price_purchase,group_id,date,day_of_week,recency,expected_purchases_30,purchase_month,most_common_day,number_of_purchases,...,purchase_trend,max_purchase,mean_purchase,median_purchase,rel_purchase_value,time_frame_char,year,month,day,label
0,9001601666,342000.0,100,2022-03-10,Thursday,96.0,0.484710,2022-03,Friday,3,...,decreasing--,342000.0,173500.000000,148500.0,-0.912281,steady,2022,3,10,0
1,9001601666,148500.0,500,2022-06-03,Friday,96.0,0.484710,2022-06,Friday,3,...,decreasing--,342000.0,173500.000000,148500.0,-0.912281,steady,2022,6,3,0
2,9001601666,30000.0,600,2022-06-14,Tuesday,96.0,0.484710,2022-06,Friday,3,...,decreasing--,342000.0,173500.000000,148500.0,-0.912281,steady,2022,6,14,0
3,9002503077,108500.0,100,2022-05-26,Thursday,44.0,1.163736,2022-05,Friday,4,...,stable,158500.0,104500.000000,105000.0,-0.064516,within-limits,2022,5,26,0
4,9002503077,158500.0,150,2022-06-10,Friday,44.0,1.163736,2022-06,Friday,4,...,stable,158500.0,104500.000000,105000.0,-0.064516,within-limits,2022,6,10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
680795,9999912003,6500.0,600,2024-06-11,Tuesday,149.0,0.770284,2024-06,Sunday,6,...,decreasing--,159500.0,100166.666667,94500.0,-0.928177,steady,2024,6,11,0
680796,9999979878,19500.0,600,2024-02-09,Friday,154.0,0.588324,2024-02,Friday,4,...,increasing++,350500.0,121787.500000,58575.0,3.776923,altering,2024,2,9,0
680797,9999979878,350500.0,190,2024-04-19,Friday,154.0,0.588324,2024-04,Friday,4,...,increasing++,350500.0,121787.500000,58575.0,3.776923,altering,2024,4,19,0
680798,9999979878,24000.0,600,2024-05-10,Friday,154.0,0.588324,2024-05,Friday,4,...,increasing++,350500.0,121787.500000,58575.0,3.776923,altering,2024,5,10,0


In [23]:
# concat_df = pd.merge(concat_df, final_df[['customers_id', 'final_cluster']], on='customers_id', how='left')

In [24]:
# concat_df['customers_id'].nunique()

82160

In [19]:
set(df_valid_labled['label'])

{0, 1}

<span style="color: red; font-size: 25px; font-weight: bold;">data shuffling</span>

In [20]:
df_valid_labled = df_valid_labled.sample(frac=1, random_state = 42).reset_index(drop=True)

<span style="color: red; font-size: 25px; font-weight: bold;">defining X and y</span>

In [21]:
X = df_valid_labled.drop(['label'], axis = 1)
y = df_valid_labled['label']
X.shape, y.shape 

((177390, 38), (177390,))

In [22]:
y.value_counts()

label
0    97756
1    79634
Name: count, dtype: int64

## <span style="color: red; font-size: 25px; font-weight: bold;">Transforming categorical data using LabelEncoder</span>


In [23]:
object_columns = X.select_dtypes(include=['object'])
object_column_names = object_columns.columns.tolist()
print(object_column_names)

['customers_id', 'group_id', 'day_of_week', 'most_common_day', 'freq_class', 'purchase_trend', 'time_frame_char']


In [24]:
from sklearn.preprocessing import LabelEncoder

# Initialize label encoders for each categorical column
label_encoder_freq_class = LabelEncoder()
label_encoder_time_frame_char = LabelEncoder()
label_encoder_purchase_trend = LabelEncoder()
label_encoder_day_of_week = LabelEncoder()
label_encoder_most_common_day = LabelEncoder()

# Fit the label encoders and transform the data
X['freq_class_encoded'] = label_encoder_freq_class.fit_transform(X['freq_class'])
X['time_frame_char_encoded'] = label_encoder_time_frame_char.fit_transform(X['time_frame_char'])
X['purchase_trend_encoded'] = label_encoder_purchase_trend.fit_transform(X['purchase_trend'])
X['day_of_week_encoded'] = label_encoder_day_of_week.fit_transform(X['day_of_week'])
X['most_common_day_encoded'] = label_encoder_most_common_day.fit_transform(X['most_common_day'])

# Drop the original categorical columns
X = X.drop(['freq_class', 'time_frame_char', 'purchase_trend', 'day_of_week', 'most_common_day'], axis=1)

# Drop unneccessary features like reshholds etc. 
X = X.drop(['tresh_1', 'tresh_2', 'tresh_3', 'd', 'purchase_month', 'date'], axis=1)

# Output the shape of X_train and y_train to ensure consistency
X.shape, y.shape

((177390, 32), (177390,))

In [25]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 177390 entries, 0 to 177389
Data columns (total 32 columns):
 #   Column                                Non-Null Count   Dtype  
---  ------                                --------------   -----  
 0   customers_id                          177390 non-null  object 
 1   price_purchase                        177390 non-null  float64
 2   group_id                              177390 non-null  object 
 3   recency                               177390 non-null  float64
 4   expected_purchases_30                 177390 non-null  float64
 5   number_of_purchases                   177390 non-null  int64  
 6   weighted_mean_time_between_purchases  177390 non-null  float64
 7   std_between_purchase                  177390 non-null  float64
 8   max_time_without_purchase             177390 non-null  float64
 9   time_since_last_purchase              177390 non-null  int64  
 10  time_since_first_purchase             177390 non-null  int64  
 11  

<span style="color: red; font-size: 25px; font-weight: bold;">scaling</span>

In [26]:
from sklearn.preprocessing import MinMaxScaler

# Specify the columns you don't want to scale
exclude_columns = ['customers_id', 'group_id']

# Create a copy of X to hold the scaled values
X_scaled = X.copy()

# Apply MinMaxScaler to the selected columns
scaler = MinMaxScaler()
columns_to_scale = [col for col in X.columns if col not in exclude_columns]

# Scale only the columns that are not in the exclude list
X_scaled[columns_to_scale] = scaler.fit_transform(X[columns_to_scale])

# X_scaled now contains the scaled columns along with the unchanged customer_id and group_id columns
X_scaled = pd.DataFrame(X_scaled, columns=X.columns)

In [28]:
from sklearn.cluster import KMeans

# Create X_clus dataframe with only the specified attributes for clustering
X_clus = X_scaled[['number_of_purchases', 'weighted_mean_time_between_purchases',
            'std_between_purchase', 'time_since_last_purchase', 'binned_purchase', 'max_purchase', 'month',
            f'expected_purchases_{length_of_the_label_window}', 'recency', 'time_since_first_purchase']]

# Apply KMeans to the entire dataset
kmeans = KMeans(n_clusters=9, random_state=42)
X_clus['final_cluster'] = kmeans.fit_predict(X_clus)

# Merge the 'segment' column back into the original X dataframe
X_scaled = X_scaled.join(X_clus['final_cluster'])

# <span style="color: green; font-size: 30px; font-weight: bold;">cluster_id</span>

In [29]:
from sklearn.cluster import KMeans

X_features = X_scaled[['customers_id', 'group_id']]

# Determine optimal number of clusters based on visualizations
k_optimal = 3  # Replace with the optimal number determined

# Apply K-Means algorithm
kmeans = KMeans(n_clusters=k_optimal, random_state=42, n_init=10)
X_scaled['segment_id'] = kmeans.fit_predict(X_features)

X_scaled.drop(['customers_id', 'group_id'], axis = 1, inplace = True)



In [30]:
feature_list = X_scaled.columns

In [31]:
print(feature_list)
len(feature_list)

Index(['price_purchase', 'recency', 'expected_purchases_30',
       'number_of_purchases', 'weighted_mean_time_between_purchases',
       'std_between_purchase', 'max_time_without_purchase',
       'time_since_last_purchase', 'time_since_first_purchase',
       'transaction recency', 'last_between_0_and_30',
       'last_between_30_and_60', 'last_between_60_and_90',
       'diff_last_and_penultimate', 'diff_penultimate_and_previous',
       'rolling_avg_3', 'rolling_avg_6', 'binned_purchase', 'max_purchase',
       'mean_purchase', 'median_purchase', 'rel_purchase_value', 'year',
       'month', 'day', 'freq_class_encoded', 'time_frame_char_encoded',
       'purchase_trend_encoded', 'day_of_week_encoded',
       'most_common_day_encoded', 'final_cluster', 'segment_id'],
      dtype='object')


32

In [32]:
# Step 1: Extract the 'final_cluster' column
final_cluster = X_scaled.iloc[:, 30]

# Step 2: Drop the 'final_cluster' column from its current position
X_scaled = X_scaled.drop(X_scaled.columns[30], axis=1)

# Step 3: Insert the 'final_cluster' column at the 25th position
X_scaled.insert(25, 'final_cluster', final_cluster)


In [34]:
feature_list = X_scaled.columns
print(feature_list)
len(feature_list)

Index(['price_purchase', 'recency', 'expected_purchases_30',
       'number_of_purchases', 'weighted_mean_time_between_purchases',
       'std_between_purchase', 'max_time_without_purchase',
       'time_since_last_purchase', 'time_since_first_purchase',
       'transaction recency', 'last_between_0_and_30',
       'last_between_30_and_60', 'last_between_60_and_90',
       'diff_last_and_penultimate', 'diff_penultimate_and_previous',
       'rolling_avg_3', 'rolling_avg_6', 'binned_purchase', 'max_purchase',
       'mean_purchase', 'median_purchase', 'rel_purchase_value', 'year',
       'month', 'day', 'final_cluster', 'freq_class_encoded',
       'time_frame_char_encoded', 'purchase_trend_encoded',
       'day_of_week_encoded', 'most_common_day_encoded', 'segment_id'],
      dtype='object')


32

# <span style="color: green; font-size: 30px; font-weight: bold;">building the model</span>

In [35]:
import pickle

with open("XGBoostmodel.pkl", "rb") as file:
    xgb_clf = pickle.load(file)

In [36]:
from sklearn.metrics import classification_report

y_pred = xgb_clf.predict(X_scaled)

print(classification_report(y, y_pred))

              precision    recall  f1-score   support

           0       0.78      0.65      0.71     97756
           1       0.64      0.77      0.70     79634

    accuracy                           0.70    177390
   macro avg       0.71      0.71      0.70    177390
weighted avg       0.72      0.70      0.70    177390



# <span style="color: green; font-size: 30px; font-weight: bold;">custom treshhold</span>

In [40]:
# Assuming X_val is your validation dataset
y_pred_prob = xgb_clf.predict_proba(X_scaled)[:, 1]  # Probabilities for class 1

# Define your custom threshold
custom_threshold = 0.8  

# Apply the custom threshold to the predicted probabilities
y_pred_custom = (y_pred_prob >= custom_threshold).astype(int)

print(classification_report(y, y_pred_custom))

              precision    recall  f1-score   support

           0       0.68      0.95      0.79     97756
           1       0.88      0.44      0.59     79634

    accuracy                           0.72    177390
   macro avg       0.78      0.70      0.69    177390
weighted avg       0.77      0.72      0.70    177390

